In [2]:
# install libraries
!pip -q install langchain openai chromadb tiktoken lark
!pip -q install sentence-transformers langchain-community langchain-huggingface
!pip -q install -U FlagEmbedding

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [1]:
!wget -O langchain_blog_posts.zip "https://www.dropbox.com/scl/fi/ulbt145sthizf2nazey49/langchain_blog_posts.zip?rlkey=9unhw0vukhlwacahmpnk5m591&dl=1"
!mkdir -p blog_posts
!unzip langchain_blog_posts.zip -d blog_posts

--2025-06-17 05:21:59--  https://www.dropbox.com/scl/fi/ulbt145sthizf2nazey49/langchain_blog_posts.zip?rlkey=9unhw0vukhlwacahmpnk5m591&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3e3c248c985099a7d57aa828f9.dl.dropboxusercontent.com/cd/0/inline/Crzfvb3E9GTYD0V7p_NBXy3wSrorKUSS0crrsTHnFmVgBi8WXDSIInT7Qfr69hsw3pjuiMg8Y3W6wtE-IeOSlWd-IuMLJcum7VkmU7MLKAZjj0uXBO7S0zsT2Fz7mGfpDJs/file?dl=1# [following]
--2025-06-17 05:21:59--  https://uc3e3c248c985099a7d57aa828f9.dl.dropboxusercontent.com/cd/0/inline/Crzfvb3E9GTYD0V7p_NBXy3wSrorKUSS0crrsTHnFmVgBi8WXDSIInT7Qfr69hsw3pjuiMg8Y3W6wtE-IeOSlWd-IuMLJcum7VkmU7MLKAZjj0uXBO7S0zsT2Fz7mGfpDJs/file?dl=1
Resolving uc3e3c248c985099a7d57aa828f9.dl.dropboxusercontent.com (uc3e3c248c985099a7d57aa828f9.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Parent Document Retriever
*   Return full docs from smaller chunks look up
*   Return bigger chunks for smaller chunks look up


In [2]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever

# Text splitting and Docloader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryStore
from langchain.document_loaders import TextLoader

BGE Embeddings

In [8]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
import torch
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

# Detect device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize embeddings
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': device},
    encode_kwargs=encode_kwargs
)

Data Preparation

In [4]:
loaders = [
    TextLoader('/content/blog_posts/blog.langchain.dev_announcing-langsmith_.txt'),
    TextLoader('/content/blog_posts/blog.langchain.dev_benchmarking-question-answering-over-csv-data_.txt'),
]
docs = []
for l in loaders:
    docs.extend(l.load())

In [5]:
len(docs)

2

In [6]:
docs[0]

Document(metadata={'source': '/content/blog_posts/blog.langchain.dev_announcing-langsmith_.txt'}, page_content='URL: https://blog.langchain.dev/announcing-langsmith/\nTitle: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications\n\nLangChain exists to make it as easy as possible to develop LLM-powered applications.\n\nWe started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “not enough tinkering happening.” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now (Nat agrees)–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragons game.\n\nThe blocker has now changed. While it’s easy to build a prototype of an application in ~5 lines of LangChain code, it’

1. Retrieving Full Documents rather than chunks

In [9]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# the vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents",
    embedding_function=bge_embeddings
)

# the storage layer for the parent documents
store = InMemoryStore()

full_doc_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter
)

<ipython-input-9-594472005>:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [10]:
full_doc_retriever.add_documents(docs)

In [11]:
# our
list(store.yield_keys())

['54ac2d11-53d3-4656-bf66-38d6c2add49b',
 '319e4e83-5ef3-4568-a715-f60557286976']

In [14]:
# vectorstore
sub_docs = vectorstore.similarity_search("What is LangChain?", k=2)

In [15]:
len(sub_docs)

2

In [16]:
print(sub_docs[0].page_content)

URL: https://blog.langchain.dev/announcing-langsmith/
Title: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications

LangChain exists to make it as easy as possible to develop LLM-powered applications.


In [17]:
retrieved_docs = full_doc_retriever.get_relevant_documents("what is langsmith")

<ipython-input-17-1174963657>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = full_doc_retriever.get_relevant_documents("what is langsmith")


In [18]:
len(retrieved_docs[0].page_content)

11652

In [19]:
retrieved_docs[0].page_content

'URL: https://blog.langchain.dev/announcing-langsmith/\nTitle: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications\n\nLangChain exists to make it as easy as possible to develop LLM-powered applications.\n\nWe started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “not enough tinkering happening.” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now (Nat agrees)–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragons game.\n\nThe blocker has now changed. While it’s easy to build a prototype of an application in ~5 lines of LangChain code, it’s still deceptively hard to take an application from prototype to production. The main issue that we see today 

## Retrive Larger Chunks

In [20]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# This text splitter is used to create the parent documents - The big chunks

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# the vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents",embedding_function=bge_embeddings
)

# the storage layer for parent documents
store = InMemoryStore()

In [21]:
big_chunks_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [22]:
big_chunks_retriever.add_documents(docs)

In [23]:
len(list(store.yield_keys()))

18

In [24]:
sub_docs = vectorstore.similarity_search("what is langsmith")

In [25]:
len(sub_docs)

4

In [26]:
retrieved_docs = big_chunks_retriever.get_relevant_documents("what is langsmith")

In [27]:
len(retrieved_docs)

2

In [28]:
print(retrieved_docs[0].page_content)

URL: https://blog.langchain.dev/announcing-langsmith/
Title: Announcing LangSmith, a unified platform for debugging, testing, evaluating, and monitoring your LLM applications

LangChain exists to make it as easy as possible to develop LLM-powered applications.

We started with an open-source Python package when the main blocker for building LLM-powered applications was getting a simple prototype working. We remember seeing Nat Friedman tweet in late 2022 that there was “not enough tinkering happening.” The LangChain open-source packages are aimed at addressing this and we see lots of tinkering happening now (Nat agrees)–people are building everything from chatbots over internal company documents to an AI dungeon master for a Dungeons and Dragons game.

The blocker has now changed. While it’s easy to build a prototype of an application in ~5 lines of LangChain code, it’s still deceptively hard to take an application from prototype to production. The main issue that we see today is appli

In [29]:
print(retrieved_docs[1].page_content)

Understanding what exactly the final prompt to the LLM call is (after all the prompt template formatting, this final prompt can be long and obfuscated)

Understanding what exactly is returned from the LLM call at each step (before it is post-processed or transformed in any way)

Understanding the exact sequence of calls to LLM (or other resources), and how they are chained together

Tracking token usage

Managing costs

Tracking (and debugging) latency

Not having a good dataset to evaluate their application over

Not having good metrics with which to evaluate their application

Understanding how users are interacting with the product

All of these problems have parallels in traditional software engineering. And, in response, a set of practices and tools for debugging, testing, logging, monitoring, etc. has emerged to help developers abstract away common infrastructure and focus on what really matters - building their applications. LLM application developers deserve the same.

LangSmit